# Main workflow:
* Modular implementation.  
* High level programming (layer architecture).  
* Import low level functions from a python script.  

In [1]:
##################################----This is pseudo code----##################################

### define train test datasets
raw_data = pd.read_csv("...")
raw_text_train, raw_text_test = split_raw_data(raw_data)

### Initial steps

text = initialize_text_model()    ## return a NN
text_data = get_text_data(raw_text_train)     ## return ready to train text data (from raw text to X_train, X_test text data)

text.fit(text_data["train"])    ## 
text.save("...")


image = initialize_image_model()
image_data = get_image_data()

image.fit(image_data["train"])
image.save("...")



### Train fusion model

text_data = get_text_data()
image_data = get_image_data()

image_model = initialize_image_model()
text_model = initialize_text_model()

headless_image_model = remove_classification_head(image_model)
headless_text_model = remove_classification_head(text_model)
headless_text_model.save("...")
headless_image_model.save("...")

headless_X_train_image = headless_image_model.predict(image_data["train"])
headless_X_train_text = headless_text_model.predict(text_data["train"])

X_train = concatenate(headless_X_train_text, headless_X_train_image)

fusion_model = build_fusion_model()
fusion_model.fit(X_train)
fusion_model.save("...")
# same with the test data


### Prediction

new_text_sample, new_image_sample = get_new_samples()

hl_image_model = load_model("...")
hl_text_model = load_model("...")

sample = concatenate(hl_text_model.predict(new_text_sample), hl_image_model.predict(new_image_sample))
fusion_model = load_model("...")
fusion_model.predict(sample)


NameError: name 'initialize_text_model' is not defined

# Import libraries

In [3]:
import pandas as pd

from FusionModel_tools import *

from sklearn.model_selection import train_test_split

In [8]:
## define general parameters

myseed = 123

# Initialize raw dataset
* Split entire dataset once here to avoid any issues (information leak, etc)

In [10]:
## import raw datasets: features and target
df_X = pd.read_csv('./datasets/X_train_update.csv', index_col = 0)
df_y = pd.read_csv('./datasets/Y_train_CVw08PX.csv', index_col = 0)


## train-test split raw data
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size = 0.2, \
                                                                random_state = myseed, stratify = df_y)

## merge features and targets
df_train = pd.concat([df_y_train,df_X_train], axis = 1)
df_test = pd.concat([df_y_test,df_X_test], axis = 1)


## save splitted dataframes
save(df_train, df_test, doit = False)

Datasets were not saved locally. Set doit = True to save datasets


<div class="alert alert-info">
<i class="fa fa-info-circle"></i> &emsp; 
    From now on, the <b>test dataset</b> will only be used to asses the models performance
</div>

In [12]:
## Visualize splitted datasets

print(df_train.shape)
display(df_train.head())

print(df_test.shape)
display(df_test.head())

(67932, 5)


,prdtypecode,designation,description,productid,imageid
45347,1920,Linen Lovers Series Confession Saint Valentin ...,Linge de maison Lovers Confession Série Saint ...,4004066770,1275045193
20881,2220,Collier 14/35 Black Tan,NaN,1183437294,1082390908
51967,2705,Les Vacances Du Lundi,NaN,1462151044,1268554280
79012,2583,Bâche hiver pour piscine 4 x 250m,<p>Bâche hiver pour piscine bois rectangle 4 p...,3690228573,1234768137
58343,1280,Steiff Fynn Teddy Bear Plush Animal Toy Beige,ulliPremium Quality Plush to Enjoy Everyday - ...,3470153950,1215429975


(16984, 5)


,prdtypecode,designation,description,productid,imageid
35666,2583,Spa 5 places Calios - Spalnéa - Acrylique Blan...,Dimensions : 215 x 185 x 93 cm //// Nombre et ...,1926714940,1113061652
19651,1320,New Flame Coupe Humidifier Night Light Mute Bu...,New Flame Coupe Humidifier Night Light Mute Bu...,4220470182,1319055786
50216,2060,Voice Control Calendar Thermometer Wooden Led ...,Voice ControlCalendar Thermometer Wooden LED D...,4079113221,1287738265
32733,2583,"Enrouleur Télescopique A. PRO ""NEW LINE"" Modèl...",Utilisation de l'Enrouleur Télescopique A. PRO...,228489648,958568141
67840,1560,Chaise De Bureau Inclinable Cuir Artificiel Rouge,<p>Cette luxueuse chaise de bureau inclinable ...,3929324475,1265058699


# Initialize Text model


In [ ]:
text = initialize_text_model()

text_data = get_text_data(raw_text_train)